In [14]:
dates= ["2019-01-12","2019-01-13"]

In [15]:
    import numpy as np
    import math
    from datetime import datetime, timedelta
    from fastai.imports import *
    from fastai.structured import *

In [16]:
def preprocess(df):
    import pandas as pd
    df = pd.concat([df, df['history'].str.join('|').str.get_dummies()], axis = 1)
    df = df.drop(["history","honeypot","blacklist","whitelist"], axis=1)
    df = pd.concat([df, pd.get_dummies(df.conn_state.apply(pd.Series), prefix="", prefix_sep="")], axis = 1)
    df = df.drop(["conn_state"], axis=1)
    df = df.replace('-', 0)
    rf_sample = df.drop(["src_ip_ext","src_ip","dest_ip"], axis=1)
    try:
        rf_sample["duration"] = rf_sample["duration"].astype(float)
        rf_sample["duration"] = np.log((rf_sample["duration"].astype('int'))+0.001)
    except ValueError:
        rf_sample.to_csv("error.csv")
    rf_sample["src_bytes"] = rf_sample["src_bytes"].astype(float)
    rf_sample["src_bytes"] = np.log((rf_sample["src_bytes"].astype('int'))+0.001)
    rf_sample["dest_bytes"] = rf_sample["dest_bytes"].astype(float)
    rf_sample["dest_bytes"] = np.log((rf_sample["dest_bytes"].astype('int'))+0.001)
    rf_sample["src_pkts"] = np.log((rf_sample["src_pkts"].astype('int'))+0.001)
    rf_sample["dest_pkts"] = np.log((rf_sample["dest_pkts"].astype('int'))+0.001)
    rf_sample['ts'] = pd.to_datetime(rf_sample['ts'],unit='s')
    rf_sample["src_port"] = rf_sample["src_port"].astype('category')
    rf_sample["dest_port"] = rf_sample["dest_port"].astype('category')
    rf_sample["src_port"] = rf_sample["src_port"].cat.codes
    rf_sample["dest_port"] = rf_sample["dest_port"].cat.codes
    xhr, yhr = [], []
    for i in df.ts:
        temp = (datetime.datetime.fromtimestamp(i) - timedelta(hours=-5)).strftime('%Y-%m-%d %H:%M:%S')[11:13]
        xhr.append(np.sin(2*math.pi*int(temp)/24))
        yhr.append(np.cos(2*math.pi*int(temp)/24))
    rf_sample['xhr'] = xhr
    rf_sample['yhr'] = yhr
    add_datepart(rf_sample, 'ts')
    rf_sample = class_balancer(rf_sample)
    return rf_sample

In [17]:
def class_balancer(df):
    if df.label.value_counts()[0] > df.label.value_counts()[1]:
        mal = df.loc[df['label'] == 1]
        ben = df.loc[df['label'] == 0].head(df.label.value_counts()[1])
        mdf = pd.concat([mal,ben], axis = 0)
        return mdf
    if df.label.value_counts()[1] > df.label.value_counts()[0]:
        mal = df.loc[df['label'] == 1].head(df.label.value_counts()[0])
        ben = df.loc[df['label'] == 0]
        mdf = pd.concat([mal,ben], axis = 0)
        return mdf

In [18]:
def sampler(dates):
    import os
    import random
    import pandas as pd
    for date in dates:
        input_dir = "/scratch/rk9cx/conn_log_labelled_runtime/labelled_"+date+"/"
        output_dir = "/scratch/rk9cx/conn_log_labelled_runtime/random_samples_W1-W2/train/"
        os.chdir(input_dir)
        #random sampling
        filename = "merged_" + date + ".csv"
        print("shuffling rows")
        n = sum(1 for line in open(filename)) - 1 #number of records in file (excludes header)
        s = 1000000 #desired sample size
        skip = sorted(random.sample(range(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list
        #preprocessing data
        print("file read")
        df = pd.read_csv(filename, skiprows=skip, low_memory=False)
        df = preprocess(df)
        df.to_csv(output_dir + "rs"+ date + ".csv", index = False) 
        print(date + "   done")

In [ ]:
sampler(dates)

shuffling rows
file read
